In [54]:
# Operator Learning via Kernel Methods
#
# This is mostly a script for prototyping methods

In [55]:
# Imports
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
import sys
sys.path.append('./')
from KLearn import *

In [56]:
# Define some Kernels
gamma = 0.1;
K = lambda y1,y2 : rbf_kernel(y1,y2, gamma = gamma)
S = lambda x1,x2 : (x1.T @ x2 + 1)**2 # lame, for now

In [57]:
# define some functions and their derivatives
f1 = lambda x : x;
df1 = lambda x : np.ones(np.shape(x));

f2 = lambda x : x**2;
df2 = lambda x : 2*x;

f3 = lambda x : np.sin(x);
df3 = lambda x : np.cos(x);

In [58]:
# Define phi, psi, and the collocation points
n = 100;
xphi = d2(np.linspace(0,1,num=n)).T;
phi = lambda f : d2(f(xphi));

m = 100;
ypsi = d2(np.linspace(0,1,num=m)).T;
psi = lambda f : d2(f(ypsi));

In [59]:
# Test it out
#phi(f1)

In [64]:
# Finish setting up
U = np.hstack( (phi(f1), phi(f2), phi(f3)) ); V = np.hstack( (psi(df1), psi(df2), psi(df3)) );
print(np.shape(U))
Y = ypsi;

(100, 3)


In [65]:
# LEARN!
D = KLearn(U,V,m,S,K,Y)

In [69]:
# Does it work... ?
Df2 = D(phi(f3));
np.max(np.abs(Df2(ypsi) - df3(ypsi))) # very nice...

7.371122592303436e-06